## 0. 모듈 불러오기

In [5]:
import os
import glob
from pathlib import Path

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
import math

import pickle
import cv2
import mediapipe as mp

from tensorflow.keras.preprocessing.image import ImageDataGenerator

## 1. 데이터 확인

In [28]:
#한 폴더당 중심 동작 20개씩 뽑음
select_df=pd.read_csv('C:/Users/HOME/Desktop/AI_project/KSL/DF_select.csv')
select_df

,name,fold,class,num,path,label
0,00_01.0045.jpg,00_01,1,45,C:/Users/HOME/Desktop/AI_project/KSL/KSL_rgb/0...,hi
1,00_01.0046.jpg,00_01,1,46,C:/Users/HOME/Desktop/AI_project/KSL/KSL_rgb/0...,hi
2,00_01.0047.jpg,00_01,1,47,C:/Users/HOME/Desktop/AI_project/KSL/KSL_rgb/0...,hi
3,00_01.0048.jpg,00_01,1,48,C:/Users/HOME/Desktop/AI_project/KSL/KSL_rgb/0...,hi
4,00_01.0049.jpg,00_01,1,49,C:/Users/HOME/Desktop/AI_project/KSL/KSL_rgb/0...,hi
...,...,...,...,...,...,...
3595,19_77.0072.jpg,19_77,77,72,C:/Users/HOME/Desktop/AI_project/KSL/KSL_rgb/1...,care
3596,19_77.0073.jpg,19_77,77,73,C:/Users/HOME/Desktop/AI_project/KSL/KSL_rgb/1...,care
3597,19_77.0074.jpg,19_77,77,74,C:/Users/HOME/Desktop/AI_project/KSL/KSL_rgb/1...,care
3598,19_77.0075.jpg,19_77,77,75,C:/Users/HOME/Desktop/AI_project/KSL/KSL_rgb/1...,care


In [32]:
select_df22=select_df[select_df['num']%2==1]

In [33]:
datagen = ImageDataGenerator(rescale=1. / 255, validation_split=0.25)
train_generator = datagen.flow_from_dataframe(
          dataframe=select_df22,
          batch_size=128,
          class_mode='categorical',
          shuffle=True,
          x_col='path',
          y_col='label',
          subset = 'training')

valid_generator = datagen.flow_from_dataframe(
          dataframe=select_df22,
          batch_size=128,
          class_mode='categorical',
          shuffle=True,
          x_col='path',
          y_col='label',
          subset = 'validation')

Found 1352 validated image filenames belonging to 10 classes.
Found 450 validated image filenames belonging to 10 classes.


### CNN

In [34]:
from tensorflow.keras import models, layers, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow import keras

In [ ]:
from keras.callbacks import ModelCheckpoint

In [35]:
pool_size = 2
    
model1 = Sequential([
  keras.Input(shape=(256,256,3)),
  Conv2D(filters = 30, kernel_size = (3,3), padding='same'),  
  BatchNormalization(),                                       
  Activation('relu'),
  Conv2D(filters = 30, kernel_size = (3,3), padding='same'),  
  BatchNormalization(),                                       
  Activation('relu'),                                           
  MaxPooling2D(pool_size=pool_size),                         

  Conv2D(filters = 50, kernel_size = (3,3), padding='same'),
  BatchNormalization(),
  Activation('relu'),
  Conv2D(filters = 50, kernel_size = (3,3), padding='same'),  
  BatchNormalization(),                                       
  Activation('relu'),   
  MaxPooling2D(pool_size=pool_size),                         

  Conv2D(filters = 80, kernel_size = (3,3), padding='same'),
  BatchNormalization(),
  Activation('relu'),
  Conv2D(filters = 80, kernel_size = (3,3), padding='same'),  
  BatchNormalization(),                                       
  Activation('relu'),   
  MaxPooling2D(pool_size=pool_size),                         

  Flatten(),
  Dense(50),
  Activation('relu'),                                          
  
  Dense(10, activation='softmax'),
])

model1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 256, 256, 30)      840       
                                                                 
 batch_normalization_12 (Bat  (None, 256, 256, 30)     120       
 chNormalization)                                                
                                                                 
 activation_14 (Activation)  (None, 256, 256, 30)      0         
                                                                 
 conv2d_13 (Conv2D)          (None, 256, 256, 30)      8130      
                                                                 
 batch_normalization_13 (Bat  (None, 256, 256, 30)     120       
 chNormalization)                                                
                                                                 
 activation_15 (Activation)  (None, 256, 256, 30)     

In [36]:
model1.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy']
)

In [37]:
history1= model1.fit(
    train_generator,
    epochs=15,
    validation_data=valid_generator,
    shuffle=True)

Epoch 1/15
11/11 [==============================] - 162s 15s/step - loss: 12.3550 - accuracy: 0.1021 - val_loss: 2.3025 - val_accuracy: 0.0889
Epoch 2/15
11/11 [==============================] - 265s 24s/step - loss: 2.3027 - accuracy: 0.0888 - val_loss: 2.3025 - val_accuracy: 0.1111
Epoch 3/15
11/11 [==============================] - 255s 24s/step - loss: 2.3024 - accuracy: 0.1050 - val_loss: 2.3025 - val_accuracy: 0.0889
Epoch 4/15
11/11 [==============================] - 241s 22s/step - loss: 2.3021 - accuracy: 0.1109 - val_loss: 2.3025 - val_accuracy: 0.0889
Epoch 5/15
11/11 [==============================] - 222s 21s/step - loss: 2.3018 - accuracy: 0.1109 - val_loss: 2.3025 - val_accuracy: 0.0889
Epoch 6/15
11/11 [==============================] - 246s 23s/step - loss: 2.3014 - accuracy: 0.1109 - val_loss: 2.3025 - val_accuracy: 0.0889
Epoch 7/15
 1/11 [=>............................] - ETA: 3:01 - loss: 2.3015 - accuracy: 0.1250

KeyboardInterrupt: 

In [38]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 17679522200898738672
 xla_global_id: -1]

### CNN+LSTM

In [ ]:
from keras.layers import Dense, Dropout, Flatten, Conv2D, BatchNormalization, MaxPooling2D,\
                        Permute, TimeDistributed, Bidirectional,GRU, LeakyReLU, SimpleRNN,\
LSTM, GlobalAveragePooling2D, SeparableConv2D, ZeroPadding2D, Reshape
from keras import models
from keras import layers
import tensorflow as tf

In [ ]:
Model = Sequential()

Model.add(SeparableConv2D(12,(3,3),activation="relu",
                 input_shape=(256,256,3)))
Model.add(BatchNormalization())
Model.add(MaxPooling2D((2,2)))

#
Model.add(SeparableConv2D(24,(3,3),
                 activation="relu",padding="same"))
Model.add(Dropout(0.2))
Model.add(MaxPooling2D((2,2)))

#
Model.add(TimeDistributed(Flatten()))
Model.add(Bidirectional(LSTM(64,
                                  return_sequences=True,
                                  dropout=0.5,
                                  recurrent_dropout=0.5)))
Model.add(Bidirectional(LSTM(64,
                                  return_sequences=True,
                                  dropout=0.5,
                                  recurrent_dropout=0.5)))

#
Model.add(Flatten())
Model.add(Dense(128,activation="relu"))
Model.add(Dropout(0.5))
Model.add(Dense(10,activation="sigmoid"))

In [ ]:
Model.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

In [ ]:
checkpoint_filepath = 'C:/Users/HOME/Desktop/AI_project/KSL/h5/' + 'best_model_{val_accuracy:.4f}.h5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1)

In [ ]:
history2= Model.fit(
    train_generator,
    epochs=10,
    validation_data=valid_generator,
    shuffle=True)

### VGG16
- 하려다가 실패함

In [20]:
from keras.applications.vgg16 import VGG16

In [25]:
base_model = VGG16(weights=None, include_top=True, input_shape=(256,256,3), classes=10)

In [ ]:
base_model.trainable = False

In [26]:
base_model.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

In [27]:
history3= base_model.fit(
    train_generator,
    epochs=10,
    validation_data=valid_generator,
    shuffle=True)

Epoch 1/10
22/22 [==============================] - 2312s 107s/step - loss: 2.4013 - accuracy: 0.1030 - val_loss: 2.3130 - val_accuracy: 0.0889
Epoch 2/10
 4/22 [====>.........................] - ETA: 33:32 - loss: 2.2984 - accuracy: 0.1328

KeyboardInterrupt: 

### Resnet

In [ ]:
tf.keras.applications.ResNet50(weights=None, include_top=True, input_shape=(256,256,3), classes=10)

### Mediapipe

In [ ]:
import time, os

actions = ['come', 'away', 'spin']
seq_length = 30
secs_for_action = 30

# MediaPipe hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

cap = cv2.VideoCapture(0)

created_time = int(time.time())
os.makedirs('dataset', exist_ok=True)

while cap.isOpened():
    for idx, action in enumerate(actions):
        data = []

        ret, img = cap.read()

        img = cv2.flip(img, 1)

        cv2.putText(img, f'Waiting for collecting {action.upper()} action...', org=(10, 30), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)
        cv2.imshow('img', img)
        cv2.waitKey(3000)

        start_time = time.time()

        while time.time() - start_time < secs_for_action:
            ret, img = cap.read()

            img = cv2.flip(img, 1)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            result = hands.process(img)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

            if result.multi_hand_landmarks is not None:
                for res in result.multi_hand_landmarks:
                    joint = np.zeros((21, 4))
                    for j, lm in enumerate(res.landmark):
                        joint[j] = [lm.x, lm.y, lm.z, lm.visibility]

                    # Compute angles between joints
                    v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3] # Parent joint
                    v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3] # Child joint
                    v = v2 - v1 # [20, 3]
                    # Normalize v
                    v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

                    # Get angle using arcos of dot product
                    angle = np.arccos(np.einsum('nt,nt->n',
                        v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                        v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]

                    angle = np.degrees(angle) # Convert radian to degree

                    angle_label = np.array([angle], dtype=np.float32)
                    angle_label = np.append(angle_label, idx)

                    d = np.concatenate([joint.flatten(), angle_label])

                    data.append(d)

                    mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)

            cv2.imshow('img', img)
            if cv2.waitKey(1) == ord('q'):
                break

        data = np.array(data)
        print(action, data.shape)
        np.save(os.path.join('dataset', f'raw_{action}_{created_time}'), data)

        # Create sequence data
        full_seq_data = []
        for seq in range(len(data) - seq_length):
            full_seq_data.append(data[seq:seq + seq_length])

        full_seq_data = np.array(full_seq_data)
        print(action, full_seq_data.shape)
        np.save(os.path.join('dataset', f'seq_{action}_{created_time}'), full_seq_data)
    break

In [ ]:
import cv2
import numpy as np

def Find(path) :
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = Resize(img)
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    img_hsv = cv2.fastNlMeansDenoisingColored(img_hsv,None,10,10,7,21) # 잡음 제거
    lower = np.array([0,48,80], dtype="uint8")
    upper = np.array([20,255,255], dtype="uint8")
    img_hand = cv2.inRange(img_hsv, lower, upper)

     #경계선 찾음

    contours, hierarchy = cv2.findContours(img_hand, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    

    # 가장 큰 영역 찾기

    max = 0

    maxcnt = None

    for cnt in contours :

        area = cv2.contourArea(cnt)

        if(max < area) :

            max = area

            maxcnt = cnt

 

    #maxcontours의 각 꼭지점 다각선 만들기 

    hull = cv2.convexHull(maxcnt)

 

    # img 다 0으로 만들기? 

    mask = np.zeros(img.shape).astype(img.dtype)

    color = [255, 255, 255]

    # 경계선 내부 255로 채우기

    cv2.fillPoly(mask, [maxcnt], color)

 

    img_hand = cv2.bitwise_and(img, mask)

    

    cv2.drawContours(img_hand, [maxcnt],0, (255,0,0), 3)

    cv2.drawContours(img_hand, [hull],0, (0,255,0), 3)

    

    # 이미지 보여주기

    cv2.imshow('image', img_hand)

 

    # 창 esc 끄기

    while True :

        if cv2.waitKey(0) == 27 :

            cv2.destroyWindow('image')

            break;

    return

 

def Resize(img) :

    print(img.shape)

    width = 500

    ratio = width/img.shape[1] # width * 사진 너비 = 비율

    height = int(ratio*img.shape[0]) # 비율 * 사진 높이

    

    # 축소 INTER_AREA

    # 확대 INTER_LINEAR

    resize = cv2.resize(img, dsize = (width, height), interpolation = cv2.INTER_AREA)

    # resize = cv2.resize(img, dsize = (0, 0), fx=1.5, fy=1.5, interpolation = cv2.INTER_AREA)

    print(resize.shape) 

    return resize